In [99]:
using MAT
using DynamicPolynomials, LinearAlgebra, SparseArrays

data="/home/hoanganh/Desktop/math-topics/InterruptedRelax/codes/dataPOP2" # path of data
D = matread(data*"/WeightsHousing3.mat");
W1 = D["W1"]; W2 = D["W2"]; c = D["c"]; x_bar=D["x_bar"]; y_bar=D["y_bar"]
eps = 0.1;

m1=size(W1,2)
m2=size(W2,2)
m3=size(W2,1)

5

In [100]:
y_bar

1

In [114]:
@polyvar x1[1:m1] x2[1:m2] x3[1:m3]# variables
f=(c[y_bar+1,:]-c[3,:])'*x3
g=Vector{Polynomial{true,Float64}}([])
h=Vector{Polynomial{true,Float64}}([])

pol=1.0*x1[1]

for j in 1:m2
    pol=x2[j]-sum(W1[j,r]*x1[r] for r=1:m1)
    append!(g,[pol])
    append!(h,[x2[j]*pol])
end
for j in 1:m3
    pol=x3[j]-sum(W2[j,r]*x2[r] for r=1:m2)
    append!(g,[pol])
    append!(h,[x3[j]*pol])
end

for t in 1:m1
    append!(g,[-x1[t]+x_bar[1,t]+eps])
end

m=length(g)
l=length(h)

f=f([x1;x2;x3]=>[x1+x_bar[1,:]-eps*ones(Float64,m1);x2;x3])

for j in 1:m
    g[j]=g[j]([x1;x2;x3]=>[x1+x_bar[1,:]-eps*ones(Float64,m1);x2;x3])
end

for j in 1:l
    h[j]=h[j]([x1;x2;x3]=>[x1+x_bar[1,:]-eps*ones(Float64,m1);x2;x3])
end
x=[x1;x2;x3]; n=length(x)
println("***Problem setting***")
println("Number of variables: n=",n)
println("Number of inequality constraints: m=",m)
println("Number of equality constraints: l=",l)

***Problem setting***
Number of variables: n=25
Number of inequality constraints: m=25
Number of equality constraints: l=12


In [118]:
using SparseArrays

include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=true);
#data="/home/hoanganh/Desktop/math-topics/InterruptedRelax/codes/dataPOP2" # path of data
#include(data*"/sparsePOPcliq$(u)nineq$(m)neq$(l).jl")

k=3
s=22
L=100*ones(Float64,100)

@time opt_val1,opt_sol=InterRelax.RelaxSparse_without_multiplier(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,s,k,assign="min",alg="MD",minimize=true,solver="Mosek",comp_opt_sol=false,L=L);

**Interrupted relaxation based on Handelman's Positivstellensatz**
Relaxation order: k=3
Sparsity order: s=22
------------------------------------------------------
The clique sizes of varibles:
[20, 8]
[1, 5]
------------------------------------------------------
  Number of cliques: p=6
  Largest clique size: u=20
 Maximal matrix size:21
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1996            
  Cones                  : 0               
  Scalar variables       : 7275            
  Matrix variables       : 97              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00     

In [119]:
using SparseArrays

include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=true);


k=1
s=22
d=Int64(maximum([sum(supp_f[:,i]) for i in 1:lmon_f]))+1

@time opt_val1,opt_sol=InterRelax.RelaxSparse(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,s,d,assign="min",alg="MD",minimize=true,solver="Mosek",comp_opt_sol=false);

**Interrupted relaxation based on Putinar-Vasilescu's Positivstellensatz**
Relaxation order: k=1
Sparsity order: s=22
Sparsity order: d=2
------------------------------------------------------
The clique sizes of varibles:
[20, 8]
[1, 5]
------------------------------------------------------
  Number of cliques: p=6
  Largest clique size: u=20
 Maximal matrix size:21
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 2872            
  Cones                  : 0               
  Scalar variables       : 6779            
  Matrix variables       : 91              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time  

In [120]:
using SparseArrays, TSSOS

include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=true);

k=2

vars,pop=InterRelax.get_POP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)
@time opt,sol,data=TSSOS.cs_tssos_first(pop,vars,k,numeq=l,CS="MD",TS=false,solver="Mosek");

************************TSSOS************************
TSSOS is launching...
------------------------------------------------------
The clique sizes of varibles:
[20, 8]
[1, 5]
------------------------------------------------------
Obtained the variable cliques in 0.000444495 seconds. The maximal size of cliques is 20.
Assembling the SDP...
There are 11451 affine constraints.
SDP assembling time: 0.115760504 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 11451           
  Cones                  : 0               
  Scalar variables       : 1843            
  Matrix variables       : 56              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator termi

In [105]:
binomial(20+2,2)

231

In [117]:
using SparseArrays, TSSOS

include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=true);

k=1

vars,pop=InterRelax.get_POP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)
@time opt,sol,data=TSSOS.cs_tssos_first(pop,vars,k,numeq=l,CS="MD",TS=false,solver="Mosek");

************************TSSOS************************
TSSOS is launching...
------------------------------------------------------
The clique sizes of varibles:
[8]
[18]
------------------------------------------------------
Obtained the variable cliques in 0.000388977 seconds. The maximal size of cliques is 8.
Assembling the SDP...
There are 198 affine constraints.
SDP assembling time: 0.001123181 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 198             
  Cones                  : 0               
  Scalar variables       : 63              
  Matrix variables       : 18              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
El

In [107]:
binomial(20+1,1)

21